In [28]:
#!/usr/bin/python3.7

In [29]:
import numpy as np
import pandas as pd

import time
import sklearn
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
from baggingPU import BaggingClassifierPU

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [30]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    columnwidth = max([len(x) for x in labels]) + 4
    empty_cell = " " * columnwidth
    print("    " + empty_cell, end=' ')
    for label in labels:
        print("%{0}s".format(columnwidth) % 'pred_' + label, end=" ")
    print()

    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % 'true_' + label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            if cell:
                print(cell, end=" ")
        print()

In [3]:
n_input = 10
n_sample = int(1e5)


df_raw = pd.read_csv('../data/1-place3.csv')

df_X = df_raw.iloc[:,:-1]
df_y = df_raw.iloc[:,-1]


print(df_X)
print(df_y)

       p1     p2
0       1      0
1       1      3
2       1      6
3       1      9
4       1     12
...    ..    ...
29994   1  29992
29995   1  29993
29996   1  29995
29997   1  29996
29998   1  29998

[29999 rows x 2 columns]
0        1
1        1
2        1
3        1
4        1
        ..
29994    0
29995    0
29996    0
29997    0
29998    0
Name: label, Length: 29999, dtype: int64


In [4]:
def transformer(x):
    x_raw = list(str(x))
    x = [0] * (n_input - len(x_raw)) + x_raw
    return pd.Series(x)

In [5]:
a = df_X.iloc[10,-1]
b = transformer(a).values.astype(np.int)
print(b)

[0 0 0 0 0 0 0 0 3 0]


In [6]:
colums = []
col_df_raw = df_X.columns.values.tolist()
i = 1
for j in range(len(b)):
    str1 = col_df_raw[i] + '-' + str(j)
    colums.append(str1)
print(colums)

['p2-0', 'p2-1', 'p2-2', 'p2-3', 'p2-4', 'p2-5', 'p2-6', 'p2-7', 'p2-8', 'p2-9']


In [7]:
df_new = pd.DataFrame(columns = colums)
print(df_new)

Empty DataFrame
Columns: [p2-0, p2-1, p2-2, p2-3, p2-4, p2-5, p2-6, p2-7, p2-8, p2-9]
Index: []


In [12]:
for i in range(29999):
    a = df_X.iloc[i,1]
    b = transformer(a).values.astype(np.int)
    se = pd.Series(b, colums)
    df_new.loc[i] = se
print(df_new)

      p2-0 p2-1 p2-2 p2-3 p2-4 p2-5 p2-6 p2-7 p2-8 p2-9
0        0    0    0    0    0    0    0    0    0    0
1        0    0    0    0    0    0    0    0    0    3
2        0    0    0    0    0    0    0    0    0    6
3        0    0    0    0    0    0    0    0    0    9
4        0    0    0    0    0    0    0    0    1    2
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
29994    0    0    0    0    0    2    9    9    9    2
29995    0    0    0    0    0    2    9    9    9    3
29996    0    0    0    0    0    2    9    9    9    5
29997    0    0    0    0    0    2    9    9    9    6
29998    0    0    0    0    0    2    9    9    9    8

[29999 rows x 10 columns]


In [13]:
df_final = pd.concat([df_X.iloc[:,0],df_new],axis=1)
print(df_final)

       p1 p2-0 p2-1 p2-2 p2-3 p2-4 p2-5 p2-6 p2-7 p2-8 p2-9
0       1    0    0    0    0    0    0    0    0    0    0
1       1    0    0    0    0    0    0    0    0    0    3
2       1    0    0    0    0    0    0    0    0    0    6
3       1    0    0    0    0    0    0    0    0    0    9
4       1    0    0    0    0    0    0    0    0    1    2
...    ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
29994   1    0    0    0    0    0    2    9    9    9    2
29995   1    0    0    0    0    0    2    9    9    9    3
29996   1    0    0    0    0    0    2    9    9    9    5
29997   1    0    0    0    0    0    2    9    9    9    6
29998   1    0    0    0    0    0    2    9    9    9    8

[29999 rows x 11 columns]


In [14]:
from sklearn.decomposition import PCA
dffinal = np.array(df_final)
print(dffinal)

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 3]
 [1 0 0 ... 0 0 6]
 ...
 [1 0 0 ... 9 9 5]
 [1 0 0 ... 9 9 6]
 [1 0 0 ... 9 9 8]]


In [15]:
pca = PCA(n_components=4)

dfraw4D = pca.fit_transform(dffinal[:,:]) 
print(dffinal)
print(dfraw4D)

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 3]
 [1 0 0 ... 0 0 6]
 ...
 [1 0 0 ... 9 9 5]
 [1 0 0 ... 9 9 6]
 [1 0 0 ... 9 9 8]]
[[-1.70053140e-11  2.42258833e-13  4.24852119e-12 -8.99966041e+00]
 [ 1.25782604e+00 -1.82951616e+00 -1.34934957e+00 -7.49966041e+00]
 [ 2.51565208e+00 -3.65903233e+00 -2.69869914e+00 -5.99966041e+00]
 ...
 [-1.67710139e+00  2.43935488e+00  1.79913276e+00  7.00026043e+00]
 [-1.25782604e+00  1.82951616e+00  1.34934957e+00  7.50026043e+00]
 [-4.19275347e-01  6.09838721e-01  4.49783190e-01  8.50026043e+00]]


In [16]:
print(pca.explained_variance_ratio_)

[0.2450694 0.2450694 0.2450694 0.2449892]


In [17]:
df_aa = pd.DataFrame(dfraw4D)
print(df_aa)

                  0             1             2        3
0     -1.700531e-11  2.422588e-13  4.248521e-12 -8.99966
1      1.257826e+00 -1.829516e+00 -1.349350e+00 -7.49966
2      2.515652e+00 -3.659032e+00 -2.698699e+00 -5.99966
3      3.773478e+00 -5.488548e+00 -4.048049e+00 -4.49966
4      6.765090e-01 -5.419122e-01 -1.413742e+00 -7.49966
...             ...           ...           ...      ...
29994 -2.934927e+00  4.268871e+00  3.148482e+00  5.50026
29995 -2.515652e+00  3.659032e+00  2.698699e+00  6.00026
29996 -1.677101e+00  2.439355e+00  1.799133e+00  7.00026
29997 -1.257826e+00  1.829516e+00  1.349350e+00  7.50026
29998 -4.192753e-01  6.098387e-01  4.497832e-01  8.50026

[29999 rows x 4 columns]


In [18]:
#use PCA

x_data = df_aa.iloc[:,:].copy()
y_data = df_y.iloc[:].copy()

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

x_train_input = pd.concat([x_train,y_train],axis=1)
x_test_input = pd.concat([x_test,y_test],axis=1)

print(len(x_train_input))
print(len(x_test_input))

print(x_train_input.label.value_counts())
print('Has null values', x_train_input.isnull().values.any())

print(x_test_input.label.value_counts())
print('Has null values', x_test_input.isnull().values.any())

23999
6000
0    15943
1     8056
Name: label, dtype: int64
Has null values False
0    4056
1    1944
Name: label, dtype: int64
Has null values False


In [15]:
#do not use PCA
x_data = df_final.iloc[:,:].copy()
y_data = df_y.iloc[:].copy()

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

x_train_input = pd.concat([x_train,y_train],axis=1)
x_test_input = pd.concat([x_test,y_test],axis=1)

print(len(x_train_input))
print(len(x_test_input))

print(x_train_input.label.value_counts())
print('Has null values', x_train_input.isnull().values.any())

print(x_test_input.label.value_counts())
print('Has null values', x_test_input.isnull().values.any())

16000
4000
1    8077
0    7923
Name: label, dtype: int64
Has null values False
0    2077
1    1923
Name: label, dtype: int64
Has null values False


In [19]:
x_train_input.head(10)

,0,1,2,3,label
27501,-3.474239,0.743583,-0.356013,-1.49974,0
18921,-1.229484,3.998665,-2.121096,-0.99970,0
2101,-1.754960,-3.000580,2.432413,-2.99966,1
8480,-2.424130,2.116927,2.095761,-2.49974,1
11552,3.424722,-4.014369,-2.481152,-0.99966,0
18821,-0.909070,0.355337,-0.217547,-3.99970,0
24444,4.154418,1.002302,-1.932926,1.00026,0
13384,-1.934507,-1.136655,-5.264570,0.50034,0
23145,-0.102531,-5.321287,3.228528,3.50030,0
7633,4.739405,2.002495,-2.743730,5.00026,1


In [20]:
df = x_train_input.copy()

NON_LBL = [c for c in df.columns if c != 'label']
X = df[NON_LBL]
y = df['label']

y_orig = y.copy()

hidden_size = 3000
y.loc[
    np.random.choice(
        y[y == 1].index, 
        replace = False, 
        size = hidden_size
    )
] = 0

In [21]:
pd.Series(y).value_counts()

0    18943
1     5056
Name: label, dtype: int64

In [22]:
print('- %d samples and %d features' % (X.shape))
print('- %d positive out of %d total before hiding labels' % (sum(df_raw.label), len(df_raw.label)))
print('- %d positive out of %d total after hiding labels' % (sum(y), len(y)))

- 23999 samples and 4 features
- 10000 positive out of 29999 total before hiding labels
- 5056 positive out of 23999 total after hiding labels


# Trainning directly

In [23]:
print(X)
print(y)

              0         1         2        3
27501 -3.474239  0.743583 -0.356013 -1.49974
18921 -1.229484  3.998665 -2.121096 -0.99970
2101  -1.754960 -3.000580  2.432413 -2.99966
8480  -2.424130  2.116927  2.095761 -2.49974
11552  3.424722 -4.014369 -2.481152 -0.99966
...         ...       ...       ...      ...
5699  -3.752648 -0.425980 -5.699665  2.50030
10742  1.257826 -1.829516 -1.349350 -5.49966
16921  1.011602  4.965950 -3.010981 -2.49970
25796  1.335685  3.610419 -1.982630  2.50026
28847 -5.711655  2.064096 -0.341325  0.00026

[23999 rows x 4 columns]
27501    0
18921    0
2101     1
8480     0
11552    0
        ..
5699     0
10742    0
16921    0
25796    0
28847    0
Name: label, Length: 23999, dtype: int64


In [24]:
print('Training MLP model ...')

from sklearn.neural_network import MLPClassifier

model = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(30,30), random_state=1,max_iter=200)

model.fit(X, y)

print('Done')

Training MLP model ...
Done


In [25]:
model.predict(X)

array([0, 0, 0, ..., 0, 0, 0])

In [26]:
print('---- {} ----'.format('MLP model'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, model.predict(X)))
print('Recall: ', recall_score(y_orig, model.predict(X)))
print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
print('f1_score: ', f1_score(y_orig, model.predict(X)))

---- MLP model ----
                        pred_negative        pred_positive 
           true_negative      15943.0          0.0 
           true_positive       8056.0          0.0 
None

Precision:  0.0
Recall:  0.0
Accuracy:  0.6643193466394434
f1_score:  0.0


# Training by bagging

In [37]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()

from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier()

from sklearn.neural_network import MLPClassifier
model3 = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(30, 30), random_state=1,max_iter=200)

from sklearn import svm
model4 = svm.LinearSVC()

from sklearn.naive_bayes import GaussianNB
model5 = GaussianNB()

from sklearn.ensemble import RandomForestClassifier
model6 = RandomForestClassifier(n_estimators = 50,n_jobs = -1)

import xgboost as xgb
model7 = xgb.XGBClassifier()

In [38]:
model_list = [model1, model2, model3, model4, model5, model6, model7]
f1 = np.zeros([len(model_list)],dtype=np.float32)

In [39]:
for i,j in zip(model_list,range(len(model_list))):
    model = BaggingClassifierPU(i,
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
    model.fit(X,y)
    print(f1_score(y_orig, model.predict(X)))
    f1[j] = f1_score(y_orig, model.predict(X))

print(f1)

0.40587210748942526
0.9893430290872617
0.42676175283850987
0.4055519625889259
0.40912406984096106
0.9627482422597755
0.9988836516993301
[0.4058721  0.98934305 0.42676175 0.40555197 0.40912408 0.9627482
 0.99888366]


In [40]:
best_model = BaggingClassifierPU(model_list[np.argmax(f1)],
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
best_model.fit(X,y)
print(best_model)

BaggingClassifierPU(base_estimator=XGBClassifier(base_score=None, booster=None,
                                                 colsample_bylevel=None,
                                                 colsample_bynode=None,
                                                 colsample_bytree=None,
                                                 gamma=None, gpu_id=None,
                                                 importance_type='gain',
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 n_estimators=1

In [41]:
print('Training bagging classifier...')
pu_start = time.perf_counter()
import xgboost as xgb
model = BaggingClassifierPU(xgb.XGBClassifier(),
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
# model = BaggingClassifierPU(MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(50), random_state=1,max_iter=100000),
#                          n_estimators = 50, 
#                          n_jobs = -1, 
#                          max_samples = sum(y)  
#                         )
model.fit(X, y)
pu_end = time.perf_counter()
print('Done!')
print('Time:', pu_end - pu_start)

Training bagging classifier...
Done!
Time: 7.44068001298001


In [42]:
#train data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, model.predict(X)))
print('Recall: ', recall_score(y_orig, model.predict(X)))
print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
print('f1_score: ', f1_score(y_orig, model.predict(X)))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative      15920.0         23.0 
           true_positive          8.0       8048.0 
None

Precision:  0.9971502911659026
Recall:  0.9990069513406157
Accuracy:  0.9987082795116463
f1_score:  0.9980777577974825


In [43]:
#print wrong predictions
y_pre = model.predict(X)
y_orig_index = y_orig.index.tolist()

FN_index = []
FT_index = []

for i in range(len(y_orig)):
    if y_orig.iloc[i] == 1 and y_pre[i] == 0 :
        FN_index.append(y_orig_index[i])
    if y_orig.iloc[i] == 0 and y_pre[i] == 1 :
        FT_index.append(y_orig_index[i])
        
print("False Negtive:")
print(X.loc[FN_index])
print("False Positive:")
print(X.loc[FT_index])

False Negtive:
             0         1         2        3
3166 -2.868250 -0.662669 -2.886869  6.00034
4000 -1.494057 -0.644856  0.593257 -7.99970
6366 -4.827429 -1.680675 -5.556191  4.00030
7000 -0.747029 -0.322428  0.296629 -8.49974
6136 -0.662846 -6.440129  1.444084  1.00030
5206  2.395788 -4.286076  1.374684  1.00030
3367  0.909070 -0.355337  0.217547 -7.99970
3536  6.292972 -3.351700  0.405717 -1.99970
False Positive:
                  0             1             2        3
23406  3.277532e-01  9.322669e-01  1.531547e-01 -7.99974
16599 -4.897949e-01 -2.545016e-01 -6.673304e-01  8.50034
29992  1.620417e-01 -6.777653e-01  5.141758e-01  8.50026
16664 -8.385507e-01  1.219677e+00  8.995664e-01  8.00034
23346 -3.240834e-01  1.355531e+00 -1.028352e+00 -7.99974
15999  7.470285e-01  3.224281e-01 -2.966285e-01  8.50034
16665 -4.192753e-01  6.098387e-01  4.497832e-01  8.50034
23259 -3.277532e-01 -9.322669e-01 -1.531547e-01  8.00030
23199 -9.795898e-01 -5.090031e-01 -1.334661e+00  8.00030
100

In [44]:
#test data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_test, model.predict(x_test)))
print('Recall: ', recall_score(y_test, model.predict(x_test)))
print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
print('f1_score: ', f1_score(y_test, model.predict(x_test)))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       4047.0          9.0 
           true_positive          2.0       1942.0 
None

Precision:  0.9953869810353665
Recall:  0.9989711934156379
Accuracy:  0.9981666666666666
f1_score:  0.997175866495507


In [45]:
#print wrong predictions
y_test_pre = model.predict(x_test)
y_test_index = y_test.index.tolist()

FN_test_index = []
FT_test_index = []

for i in range(len(y_test)):
    if y_test.iloc[i] == 1 and y_test_pre[i] == 0 :
        FN_test_index.append(y_test_index[i])
    if y_test.iloc[i] == 0 and y_test_pre[i] == 1 :
        FT_test_index.append(y_test_index[i])

print("False Negtive:")
print(x_test.loc[FN_test_index])
print("False Positive:")
print(x_test.loc[FT_test_index])

False Negtive:
             0         1         2        3
3334  0.838551 -1.219677 -0.899566 -7.99970
6667  0.419275 -0.609839 -0.449783 -8.49974
False Positive:
                  0             1             2        3
23332  1.756393e-11 -2.334591e-13 -4.492901e-12  9.00030
23340  2.572336e-01  6.792656e-02 -9.639589e-01 -7.99974
24000 -3.277532e-01 -9.322669e-01 -1.531547e-01 -7.99974
23331 -8.385507e-01  1.219677e+00  8.995664e-01  8.00030
15992  9.090702e-01 -3.553371e-01  2.175472e-01  8.00034
22599  2.572336e-01  6.792656e-02 -9.639589e-01  8.00030
10006 -1.620417e-01  6.777653e-01 -5.141758e-01 -8.49966
23319  3.240834e-01 -1.355531e+00  1.028352e+00  8.00030
16673 -1.620417e-01  6.777653e-01 -5.141758e-01 -8.49970
